# **FORMATING DATASET**

##### Melakukan penyeragaman jumlah fitur, nama fitur, tipe data, format value data pada dataset

In [150]:
import pandas as pd
import os
import numpy as np

In [151]:
CUACA_PATH = r"E:\competition\DataVidia 2026\data\01-raw\cuaca-harian"
ISP_PATH = r"E:\competition\DataVidia 2026\data\01-raw\ISPU"
JUMLAH_PENDUDUK_PATH = r"E:\competition\DataVidia 2026\data\01-raw\jumlah-penduduk"
KUALITAS_AIR_SUNGAI_PATH = r"E:\competition\DataVidia 2026\data\01-raw\kualitas-air-sungai"
LIBUR_NASIONAL_PATH = r"E:\competition\DataVidia 2026\data\01-raw\libur-nasional"
NDVI_PATH = r"E:\competition\DataVidia 2026\data\01-raw\NDVI (vegetation index)"

## **1. ISPU**

In [152]:
COLUMN_MAPPING = {
    # polutan
    'pm_sepuluh': 'pm10',
    'pm_10': 'pm10',
    'pm_duakomalima': 'pm2.5',
    'pm25': 'pm2.5',
    'sulfur_dioksida': 'so2',
    'karbon_monoksida': 'co',
    'ozon': 'o3',
    'nitrogen_dioksida': 'no2',

    # metadata
    'lokasi_spku': 'stasiun',
    'station': 'stasiun',

    # kategori
    'category': 'kategori',
    'categori': 'kategori',

    'parameter_pencemar_kritis': 'kritis',
    'critical' : 'kritis',
}

STANDARD_COLUMNS = [
    'periode_data', 'tanggal', 'stasiun',
    'pm10', 'pm2.5', 'so2', 'co', 'o3', 'no2',
    'max', 'kritis', 'kategori'
]

In [153]:
def get_csv_files(folder_path):
    """
    Fungsi untuk mengembalikan list nama file CSV dalam folder.
    """
    return [
        file for file in os.listdir(folder_path)
        if file.endswith(".csv")
    ]

def read_csv_file(folder_path, filename):
    """
    Fungsi untuk membaca file CSV dan mengembalikan DataFrame.
    """
    file_path = os.path.join(folder_path, filename)
    try:
        return pd.read_csv(file_path)
    except Exception as e:
        raise RuntimeError(f"Gagal membaca file {filename}: {e}")


def standardize_columns(df: pd.DataFrame) -> pd.DataFrame:
    """
    Fungsi untuk menyamakan nama dan struktur kolom 
    pada tabel
    """

    df = df.rename(columns=COLUMN_MAPPING)

    for col in ['bulan', 'tahun']:
        if col in df.columns:
            df = df.drop(columns=[col])

    for col in STANDARD_COLUMNS:
        if col not in df.columns:
            df[col] = np.nan

    return df[STANDARD_COLUMNS]

def merge_date_columns(df: pd.DataFrame) -> pd.DataFrame:
    """
    Fungsi untuk menggabungkan kolom tanggal, bulan, dan tahun 
    menjadi satu kolom tanggal
    """

    df["tahun"] = df["periode_data"].astype(str).str[:4].astype(int)

    df["tanggal"] = pd.to_datetime(
        dict(
            year=df["tahun"],
            month=df["bulan"],
            day=df["tanggal"]
        ),
        errors="coerce"
    )

    df.drop(columns=["tahun", "bulan"], inplace=True)
    return df

def normalize_missing_values(df: pd.DataFrame) -> pd.DataFrame:
    """
    Fungsi untuk mengubah simbol-simbol yang digunakan
    untuk menandai nilai hilang menjadi NaN
    """

    missing_symbols = [
        "-", "--", "---", "—", "–",
        "N/A", "NA", "n/a",
        "null", "NULL", "None", "none",
        "", " "
    ]
    
    return df.replace(missing_symbols, np.nan)

def normalize_date(df: pd.DataFrame) -> pd.DataFrame:
    """
    Normalisasi kolom 'tanggal' menjadi datetime (YYYY-MM-DD)
    """
    if "time" in df.columns and "tanggal" not in df.columns:
        df = df.rename(columns={"time": "tanggal"})

    if "tanggal" not in df.columns:
        return df

    tanggal = df["tanggal"]

    # Kalau sudah datetime, langsung return
    if pd.api.types.is_datetime64_any_dtype(tanggal):
        return df

    tanggal_str = tanggal.astype(str)
    parsed_tanggal = pd.Series(pd.NaT, index=df.index)

    date_formats = {
        r"\d{4}-\d{2}-\d{2}": "%Y-%m-%d",
        r"\d{4}/\d{2}/\d{2}": "%Y/%m/%d",
        r"\d{2}/\d{2}/\d{4}": "%d/%m/%Y",
    }

    for pattern, fmt in date_formats.items():
        mask = tanggal_str.str.match(pattern, na=False)
        parsed_tanggal[mask] = pd.to_datetime(
            tanggal_str[mask],
            format=fmt,
            errors="coerce"
        )

    fallback_mask = parsed_tanggal.isna()
    parsed_tanggal[fallback_mask] = pd.to_datetime(
        tanggal_str[fallback_mask],
        errors="coerce"
    )

    df["tanggal"] = parsed_tanggal
    return df

def convert_to_numeric(df: pd.DataFrame, except_cols: list | str = "time") -> pd.DataFrame:
    """
    Fungsi untuk mengubah semua kolom kecuali kolom tertentu menjadi numeric.
    """
    # pastikan except_cols selalu list
    if isinstance(except_cols, str):
        except_cols = [except_cols]

    for col in df.columns:
        if col not in except_cols:
            df[col] = pd.to_numeric(df[col], errors='coerce')
    return df

def uppercase_string_values(df: pd.DataFrame) -> pd.DataFrame:
    """
    Fungsi untuk mengubah semua value bertipe string menjadi UPPER CASE
    dan menghapus simbol/tanda baca (diganti spasi)
    """
    for col in df.columns:
        if pd.api.types.is_object_dtype(df[col]) or pd.api.types.is_string_dtype(df[col]):
            df[col] = (
                df[col]
                .str.upper()
                # ganti semua selain huruf & angka jadi spasi
                .str.replace(r"[^A-Z0-9]", " ", regex=True)
                # rapikan spasi berlebih
                .str.replace(r"\s+", " ", regex=True)
                .str.strip()
            )
    return df

def extract_station_code(df: pd.DataFrame, col: str = "stasiun") -> pd.DataFrame:
    """
    Fungsi untuk mengambil kode stasiun (kata pertama) dari kolom stasiun.
    Contoh: 'DKI5 Kebon Jeruk Jakarta Barat' -> 'DKI5'
    """
    if col not in df.columns:
        return df

    df[col] = (
        df[col]
        .astype(str)
        .str.strip()
        .str.split()
        .str[0]
    )

    return df


In [154]:
output_path = r"E:\competition\DataVidia 2026\data\02-cleaned\ISPU"

for file in get_csv_files(ISP_PATH):
    df = read_csv_file(ISP_PATH, file)

    # merge tanggal hanya untuk 2024 & 2025
    if ("2024" in file or "2025" in file) and \
       {"periode_data", "bulan", "tanggal"}.issubset(df.columns):
        df = merge_date_columns(df)

    df = standardize_columns(df)
    df = normalize_date(df)
    df = normalize_missing_values(df)
    df['periode_data'] = df['periode_data'].astype(str)
    df = convert_to_numeric(df, except_cols=['periode_data', 'tanggal', 'stasiun', 'kategori', 'kritis'])
    df = uppercase_string_values(df)
    df = extract_station_code(df, col="stasiun")

    output_file = os.path.join(output_path, file)
    df.to_csv(output_file, index=False)

    print(f"Selesai: {file}")

Selesai: data-indeks-standar-pencemar-udara-(ispu)-di-provinsi-dki-jakarta-2023-komponen-data.csv
Selesai: indeks-standar-pencemaran-udara-(ispu)-tahun-2010-komponen-data.csv
Selesai: indeks-standar-pencemaran-udara-(ispu)-tahun-2011-komponen-data.csv
Selesai: indeks-standar-pencemaran-udara-(ispu)-tahun-2012-komponen-data.csv
Selesai: indeks-standar-pencemaran-udara-(ispu)-tahun-2013-komponen-data.csv
Selesai: indeks-standar-pencemaran-udara-(ispu)-tahun-2014-komponen-data.csv
Selesai: indeks-standar-pencemaran-udara-(ispu)-tahun-2015-komponen-data.csv
Selesai: indeks-standar-pencemaran-udara-(ispu)-tahun-2016-komponen-data.csv
Selesai: indeks-standar-pencemaran-udara-(ispu)-tahun-2017-komponen-data.csv
Selesai: indeks-standar-pencemaran-udara-(ispu)-tahun-2018-komponen-data.csv
Selesai: indeks-standar-pencemaran-udara-(ispu)-tahun-2019-komponen-data.csv
Selesai: indeks-standar-pencemaran-udara-(ispu)-tahun-2020-komponen-data.csv
Selesai: indeks-standar-pencemaran-udara-(ispu)-tahun-2

In [155]:
df_ispu = pd.read_csv(r"E:\competition\DataVidia 2026\data\02-cleaned\ISPU\data-indeks-standar-pencemar-udara-(ispu)-di-provinsi-dki-jakarta-2023-komponen-data.csv")
df_ispu.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1825 entries, 0 to 1824
Data columns (total 12 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   periode_data  1825 non-null   int64  
 1   tanggal       1825 non-null   object 
 2   stasiun       1825 non-null   object 
 3   pm10          1603 non-null   float64
 4   pm2.5         1530 non-null   float64
 5   so2           1786 non-null   float64
 6   co            1781 non-null   float64
 7   o3            1796 non-null   float64
 8   no2           1759 non-null   float64
 9   max           1822 non-null   float64
 10  kritis        1804 non-null   object 
 11  kategori      1825 non-null   object 
dtypes: float64(7), int64(1), object(4)
memory usage: 171.2+ KB


In [156]:
df_ispu.head()

,periode_data,tanggal,stasiun,pm10,pm2.5,so2,co,o3,no2,max,kritis,kategori
0,202302,2023-02-25,DKI5,35.0,NaN,13.0,12.0,31.0,18.0,35.0,PM10,BAIK
1,202302,2023-02-26,DKI5,23.0,NaN,14.0,9.0,32.0,11.0,32.0,O3,BAIK
2,202302,2023-02-27,DKI5,20.0,NaN,13.0,8.0,33.0,13.0,33.0,O3,BAIK
3,202302,2023-02-28,DKI5,30.0,NaN,21.0,11.0,28.0,18.0,30.0,PM10,BAIK
4,202303,2023-03-01,DKI1,38.0,44.0,50.0,8.0,19.0,27.0,50.0,3,BAIK


## **2. cuaca-harian**

In [157]:
output_path = r"E:\competition\DataVidia 2026\data\02-cleaned\cuaca-harian"

for file in get_csv_files(CUACA_PATH):
    df = read_csv_file(CUACA_PATH, file)

    df = normalize_date(df)
    df = normalize_missing_values(df)
    df = convert_to_numeric(df, except_cols='tanggal')

    output_file = os.path.join(output_path, file)
    df.to_csv(output_file, index=False)

    print(f"Selesai: {file}")

Selesai: cuaca-harian-dki1-bundaranhi.csv
Selesai: cuaca-harian-dki2-kelapagading.csv
Selesai: cuaca-harian-dki3-jagakarsa.csv
Selesai: cuaca-harian-dki4-lubangbuaya.csv
Selesai: cuaca-harian-dki5-kebonjeruk.csv


In [158]:
df_cuaca = pd.read_csv(r"E:\competition\DataVidia 2026\data\02-cleaned\cuaca-harian\cuaca-harian-dki3-jagakarsa.csv")
df_cuaca.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5722 entries, 0 to 5721
Data columns (total 24 columns):
 #   Column                           Non-Null Count  Dtype  
---  ------                           --------------  -----  
 0   tanggal                          5722 non-null   object 
 1   temperature_2m_max (°C)          5722 non-null   float64
 2   temperature_2m_min (°C)          5722 non-null   float64
 3   precipitation_sum (mm)           5722 non-null   float64
 4   precipitation_hours (h)          5722 non-null   float64
 5   wind_speed_10m_max (km/h)        5722 non-null   float64
 6   wind_direction_10m_dominant (°)  5722 non-null   int64  
 7   shortwave_radiation_sum (MJ/m²)  5722 non-null   float64
 8   temperature_2m_mean (°C)         5722 non-null   float64
 9   relative_humidity_2m_mean (%)    5722 non-null   int64  
 10  cloud_cover_mean (%)             5722 non-null   int64  
 11  surface_pressure_mean (hPa)      5722 non-null   float64
 12  wind_gusts_10m_max (

In [159]:
df_cuaca.head()

,tanggal,temperature_2m_max (°C),temperature_2m_min (°C),precipitation_sum (mm),precipitation_hours (h),wind_speed_10m_max (km/h),wind_direction_10m_dominant (°),shortwave_radiation_sum (MJ/m²),temperature_2m_mean (°C),relative_humidity_2m_mean (%),...,relative_humidity_2m_max (%),relative_humidity_2m_min (%),cloud_cover_max (%),cloud_cover_min (%),wind_gusts_10m_mean (km/h),wind_speed_10m_mean (km/h),wind_gusts_10m_min (km/h),wind_speed_10m_min (km/h),surface_pressure_max (hPa),surface_pressure_min (hPa)
0,2010-01-01,29.8,23.7,4.0,14.0,16.0,246,16.24,26.1,85,...,94,71,100,99,21.0,10.5,11.9,6.9,1001.8,997.6
1,2010-01-02,30.9,22.8,6.5,4.0,14.7,238,19.80,26.3,85,...,97,69,100,91,16.5,7.7,9.0,4.4,1002.3,998.5
2,2010-01-03,30.5,23.5,7.6,11.0,12.6,244,17.32,26.2,86,...,95,69,100,81,18.4,9.4,11.9,6.5,1003.0,999.0
3,2010-01-04,30.1,24.4,0.9,5.0,19.3,239,20.43,26.9,83,...,94,71,100,17,23.8,13.5,14.4,9.6,1001.6,997.7
4,2010-01-05,30.5,24.2,14.3,7.0,15.9,247,15.86,26.1,85,...,92,70,100,99,21.6,11.1,10.4,7.8,1001.5,998.5


## **3. jumlah-penduduk**

In [160]:
output_path = r"E:\competition\DataVidia 2026\data\02-cleaned\jumlah-penduduk"

for file in get_csv_files(JUMLAH_PENDUDUK_PATH):
    df = read_csv_file(JUMLAH_PENDUDUK_PATH, file)

    df = normalize_missing_values(df)
    df = convert_to_numeric(df, except_cols=['periode_data', 'nama_provinsi', 'nama_kabupaten_kota', 'nama_kecamatan', 'nama_kelurahan', 'jenis_kelamin'])
    df = uppercase_string_values(df)

    output_file = os.path.join(output_path, file)
    df.to_csv(output_file, index=False)

    print(f"Selesai: {file}")

Selesai: data-jumlah-penduduk-provinsi-dki-jakarta-berdasarkan-kelompok-usia-dan-jenis-kelamin-tahun-2013-2021-komponen-data.csv


In [161]:
df_jpenduduk = pd.read_csv(r"E:\competition\DataVidia 2026\data\02-cleaned\jumlah-penduduk\data-jumlah-penduduk-dki-jakarta-cleaned.csv")
df_jpenduduk.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 34176 entries, 0 to 34175
Data columns (total 9 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   periode_data         34176 non-null  int64 
 1   tahun                34176 non-null  int64 
 2   nama_provinsi        34176 non-null  object
 3   nama_kabupaten_kota  34176 non-null  object
 4   nama_kecamatan       34176 non-null  object
 5   nama_kelurahan       34176 non-null  object
 6   usia                 34176 non-null  object
 7   jenis_kelamin        34176 non-null  object
 8   jumlah_penduduk      34176 non-null  int64 
dtypes: int64(3), object(6)
memory usage: 2.3+ MB


In [162]:
df_jpenduduk.head()

,periode_data,tahun,nama_provinsi,nama_kabupaten_kota,nama_kecamatan,nama_kelurahan,usia,jenis_kelamin,jumlah_penduduk
0,2013,2013,PROVINSI DKI JAKARTA,JAKARTA TIMUR,JATINEGARA,KAMPUNG MELAYU,0-4,LAKI-LAKI,1419
1,2013,2013,PROVINSI DKI JAKARTA,JAKARTA SELATAN,PESANGGRAHAN,BINTARO,15-19,LAKI-LAKI,1981
2,2013,2013,PROVINSI DKI JAKARTA,JAKARTA SELATAN,PESANGGRAHAN,PESANGGRAHAN,15-19,LAKI-LAKI,1084
3,2013,2013,PROVINSI DKI JAKARTA,JAKARTA SELATAN,JAGAKARSA,CIPEDAK,15-19,LAKI-LAKI,1533
4,2013,2013,PROVINSI DKI JAKARTA,JAKARTA SELATAN,JAGAKARSA,TANJUNG BARAT,15-19,LAKI-LAKI,1691


## **4. kualitas-air-sungai**

In [163]:
def merge_year_and_month(
    df: pd.DataFrame,
    year_col: str = "periode_data",
    month_col: str = "bulan_sampling"
) -> pd.DataFrame:
    """
    Fungsi untuk menggabungkan tahun (periode_data) dan bulan_sampling
    menjadi satu kolom periode_data (YYYYMM).
    """

    if year_col not in df.columns or month_col not in df.columns:
        return df

    year = pd.to_numeric(df[year_col], errors="coerce")
    month = pd.to_numeric(df[month_col], errors="coerce")

    month_str = month.astype("Int64").astype(str).str.zfill(2)

    df[year_col] = (
        year.astype("Int64").astype(str) + month_str
    ).astype("Int64")
    
    df.drop(columns=[month_col], inplace=True)

    return df

In [164]:
output_path = r"E:\competition\DataVidia 2026\data\02-cleaned\kualitas-air-sungai"

for file in get_csv_files(KUALITAS_AIR_SUNGAI_PATH):
    df = read_csv_file(KUALITAS_AIR_SUNGAI_PATH, file)

    df = merge_year_and_month(df, year_col='periode_data', month_col='bulan_sampling')
    df = normalize_missing_values(df)
    df = uppercase_string_values(df)

    output_file = os.path.join(output_path, file)
    df.to_csv(output_file, index=False)

    print(f"Selesai: {file}")

Selesai: data-kualitas-air-sungai-komponen-data.csv


In [165]:
df_kual_air = pd.read_csv(r"E:\competition\DataVidia 2026\data\02-cleaned\kualitas-air-sungai\data-kualitas-air-sungai-komponen-data.csv")
df_kual_air.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14400 entries, 0 to 14399
Data columns (total 11 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   periode_data        14400 non-null  int64  
 1   periode_pemantauan  14400 non-null  object 
 2   titik_sampel        14400 non-null  object 
 3   nama_sungai         14400 non-null  object 
 4   alamat              14400 non-null  object 
 5   latitude            14400 non-null  float64
 6   longitude           14400 non-null  float64
 7   jenis_parameter     14400 non-null  object 
 8   parameter           14400 non-null  object 
 9   baku_mutu           14400 non-null  float64
 10  hasil_pengukuran    14400 non-null  float64
dtypes: float64(4), int64(1), object(6)
memory usage: 1.2+ MB


In [166]:
df_kual_air.head()

,periode_data,periode_pemantauan,titik_sampel,nama_sungai,alamat,latitude,longitude,jenis_parameter,parameter,baku_mutu,hasil_pengukuran
0,202405,PERIODE 2,KLT 3,KALIBARU TIMUR,JL RAYA BOGOR KOMSEKO,-6.286182,106.870626,KIMIA,PH,6.9,7.11
1,202405,PERIODE 2,KLT 3,KALIBARU TIMUR,JL RAYA BOGOR KOMSEKO,-6.286182,106.870626,KIMIA,BOD,3.0,13.00
2,202405,PERIODE 2,KLT 3,KALIBARU TIMUR,JL RAYA BOGOR KOMSEKO,-6.286182,106.870626,KIMIA,COD,25.0,41.00
3,202405,PERIODE 2,KLT 3,KALIBARU TIMUR,JL RAYA BOGOR KOMSEKO,-6.286182,106.870626,KIMIA,DO,4.0,5.60
4,202405,PERIODE 2,KLT 3,KALIBARU TIMUR,JL RAYA BOGOR KOMSEKO,-6.286182,106.870626,KIMIA,NITRAT,10.0,2.00


## **5. libur-nasional**

In [167]:
output_path = r"E:\competition\DataVidia 2026\data\02-cleaned\libur-nasional"

for file in get_csv_files(LIBUR_NASIONAL_PATH):
    df = read_csv_file(LIBUR_NASIONAL_PATH, file)

    df = normalize_date(df)
    df = normalize_missing_values(df)
    df = uppercase_string_values(df)

    df = df.rename(columns={"day_name": "nama_hari"})

    output_file = os.path.join(output_path, file)
    df.to_csv(output_file, index=False)

    print(f"Selesai: {file}")

Selesai: dataset-libur-nasional-dan-weekend.csv


In [168]:
df_libur = pd.read_csv(r"E:\competition\DataVidia 2026\data\02-cleaned\libur-nasional\dataset-libur-nasional-dan-weekend.csv")
df_libur.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5844 entries, 0 to 5843
Data columns (total 5 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   tanggal              5844 non-null   object
 1   is_holiday_nasional  5844 non-null   int64 
 2   nama_libur           257 non-null    object
 3   is_weekend           5844 non-null   int64 
 4   nama_hari            5844 non-null   object
dtypes: int64(2), object(3)
memory usage: 228.4+ KB


In [169]:
df_libur.head()

,tanggal,is_holiday_nasional,nama_libur,is_weekend,nama_hari
0,2010-01-01,1,NEW YEAR S DAY,0,FRIDAY
1,2010-01-02,0,NaN,1,SATURDAY
2,2010-01-03,0,NaN,1,SUNDAY
3,2010-01-04,0,NaN,0,MONDAY
4,2010-01-05,0,NaN,0,TUESDAY


## **6. NDVI**

In [170]:
output_path = r"E:\competition\DataVidia 2026\data\02-cleaned\NDVI"

for file in get_csv_files(NDVI_PATH):
    df = read_csv_file(NDVI_PATH, file)

    df = normalize_date(df)
    df = normalize_missing_values(df)
    df = uppercase_string_values(df)

    output_file = os.path.join(output_path, file)
    df.to_csv(output_file, index=False)

    print(f"Selesai: {file}")

Selesai: indeks-ndvi-jakarta.csv


In [171]:
df_ndvi = pd.read_csv(r"E:\competition\DataVidia 2026\data\02-cleaned\NDVI\indeks-ndvi-jakarta.csv")
df_ndvi.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1810 entries, 0 to 1809
Data columns (total 3 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   tanggal     1810 non-null   object 
 1   stasiun_id  1810 non-null   object 
 2   ndvi        1810 non-null   float64
dtypes: float64(1), object(2)
memory usage: 42.6+ KB


In [172]:
df_ndvi.head()

,tanggal,stasiun_id,ndvi
0,2009-12-19,DKI5,0.3801
1,2010-01-01,DKI5,0.3705
2,2010-01-17,DKI5,0.4848
3,2010-02-02,DKI5,0.5710
4,2010-02-18,DKI5,0.4308
